4 indicateurs d'impact data.gouv.fr
- taux de réponse "oui" à la question "Avez-vous trouvé ce que vous cherchiez ?" (non encore calculé)
- Score qualité moyen du top 1000 jeux de données (calculé dans ce notebook)
- Temps moyen de réponse à un commentaire usager (calculé dans ce notebook)
- Nombre de réutilisations de qualité sur le TOP 100 des jeux de données (calculé manuellement par l'équipe)

# Score qualité moyen du top 1000 jeux de données

Méthodologie :
- on récupère le catalogue des jeux de données data.gouv.fr, on le trie par metric.views (correspondant aux métriques matomo, à changer peut-être lors de la mise en prod de l'api metrics). On garde les 1000 premiers et on calcule le score qualité moyen.

In [2]:
import pandas as pd

In [4]:
df_datasets = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/f868cca6-8da1-4369-a78d-47463f19a9a3", dtype=str, sep=";")

In [5]:
df_datasets["metric.views"] = df_datasets["metric.views"].astype(float)

In [6]:
df_datasets = df_datasets.sort_values(by="metric.views", ascending=False)

In [7]:
final = df_datasets[:1000]

In [8]:
final["quality_score"] = final["quality_score"].astype(float)

/var/folders/cg/b2_z5vjn52x7nyrtvtmwqsqh0000gp/T/ipykernel_36154/788487950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final["quality_score"] = final["quality_score"].astype(float)


In [11]:
round(100 * final["quality_score"].mean(), 2)

70.0

# Temps moyen de réponse à un commentaire usager 

Méthodologie : 
- On récupère les discussions relatives à un dataset sur la dernière année, on regarde si il y a eu une réponse, s'il n'y en a pas eu, on applique un score de 30 jours par défaut. Si il y a eu une réponse, on regarde le temps de réponse, si plus de 30 jours, on garde 30 jours, si moins, on prend le temps effectif. On fait ensuite la moyenne de tout ces temps.

In [15]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

oneyearago = date.today() - relativedelta(years=1)
oneyearago = oneyearago.strftime("%Y-%m-%d")

In [33]:
page = 1
nb_discussions = 0
nb_discussions_with_answer = 0
time_to_answer = []
actual_date = date.today().strftime("%Y-%m-%d")
while actual_date > oneyearago:
    print(page, actual_date)
    r = requests.get("https://www.data.gouv.fr/api/1/discussions/?page_size=100&page="+str(page))
    data = r.json()
    for item in data["data"]:
        actual_date = item["discussion"][0]["posted_on"]
        if actual_date > oneyearago and item["subject"]["class"] == "Dataset":
            nb_discussions += 1
            if len(item["discussion"]) > 1:
                nb_discussions_with_answer += 1
                date_format = "%Y-%m-%dT%H:%M:%S" 
                first_date = datetime.strptime(item["discussion"][0]["posted_on"][:19], date_format)
                second_date = datetime.strptime(item["discussion"][1]["posted_on"][:19], date_format)
                ecart = second_date - first_date
                ecart_jour = ecart.days + (ecart.seconds/(3600*24))
                if ecart_jour > 30:
                    time_to_answer.append(30)
                else:
                    time_to_answer.append(ecart_jour)
            else:
                time_to_answer.append(30)
    page = page + 1

1 2023-09-11
2 2023-08-23T14:01:38.600000+00:00
3 2023-07-27T12:35:35.380000+00:00
4 2023-06-30T09:34:23.993000+00:00
5 2023-06-07T12:52:01.118000+00:00
6 2023-05-11T14:00:57.405000+00:00
7 2023-04-13T16:59:27.536000+00:00
8 2023-03-16T16:28:13.299000+00:00
9 2023-02-14T21:25:29.180000+00:00
10 2023-01-24T21:34:01.058000+00:00
11 2023-01-03T14:08:13.388000+00:00
12 2022-12-11T12:28:54.479000+00:00
13 2022-11-22T07:41:31.581000+00:00
14 2022-10-27T14:33:28.047000+00:00
15 2022-10-06T15:03:08.793000+00:00
16 2022-09-13T14:12:15.343000+00:00


In [34]:
nb_discussions

1455

In [35]:
nb_discussions_with_answer

989

In [36]:
len(time_to_answer)

1455

In [37]:
import numpy as np

In [38]:
arr = np.array(time_to_answer)

Le nombre de jours qu'on garde : 

In [40]:
round(arr.mean(), 2)

14.38

In [41]:
time_to_answer_if_answer = [x for x in time_to_answer if x < 30]

In [42]:
arr2 = np.array(time_to_answer_if_answer)

Si on ne prend pas en compte les commentaires non répondus ou au-delà de 30 jours

In [43]:
arr2.mean()

4.660037380259301